<a href="https://colab.research.google.com/github/zhestyatsky/MCL-WiC/blob/main/experiments/general_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!git clone https://github.com/zhestyatsky/MCL-WiC.git MCL-WiC

import sys
sys.path.insert(0, '/content/MCL-WiC')

Cloning into 'MCL-WiC'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 639 (delta 6), reused 9 (delta 3), pack-reused 620
Receiving objects: 100% (639/639), 5.41 MiB | 25.88 MiB/s, done.
Resolving deltas: 100% (324/324), done.


In [ ]:
!pip install --upgrade -r MCL-WiC/requirements.txt

     |████████████████████████████████| 12.8MB 231kB/s 
     |████████████████████████████████| 7.0MB 48.0MB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
     |████████████████████████████████| 696kB 53.7MB/s 
     |████████████████████████████████| 1.8MB 45.2MB/s 
     |████████████████████████████████| 1.2MB 47.1MB/s 
     |████████████████████████████████| 22.3MB 16.0MB/s 
     |████████████████████████████████| 829kB 42.4MB/s 
     |████████████████████████████████| 276kB 56.9MB/s 
     |████████████████████████████████| 112kB 53.0MB/s 
     |████████████████████████████████| 2.9MB 51.1MB/s 
     |████████████████████████████████| 870kB 42.8MB/s 
     |████████████████████████████████| 1.3MB 48.2MB/s 
     |████████████████████████████████| 143kB 59.8MB/s 
     |████████████████████████████████| 296kB 56.4MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=77e1c29dff3ecc506b98b598990defe9a64db9e6119fdf45b7a31f6dfef2939b
  Stor

In [ ]:
### THIS IS WHERE YOU SELECT THE MODEL ###
### ALL AVAILABLE MODELS ARE LISTED IN src/training/config.py ###

MODEL_DESCRIPTION = "bert-large-cos_sim-relu-only_wic"

In [ ]:
from src.data.dataset import get_train_val_test_loaders
from src.training.models import get_model
from src.training.trainer import get_trainer
from src.util.util import seed_everything

seed_everything(42)

train_loader, val_loader, test_loader = get_train_val_test_loaders(MODEL_DESCRIPTION)
clf = get_model(MODEL_DESCRIPTION)
trainer = get_trainer()

trainer.fit(clf, train_loader, val_loader)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type             | Params
----------------------------------------------------
0 | model          | BertModel        | 333 M 
1 | loss           | BCELoss          | 0     
2 | valid_accuracy | Accuracy         | 0     
3 | test_accuracy  | Accuracy         | 0     
4 | activation     | ReLU             | 0     
5 | cos            | CosineSimilarity | 0     
----------------------------------------------------
333 M     Trainable params
0         Non-trainable params
333 M     Total params


1

In [ ]:
trainer.test(clf, val_loader)


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


[{}]

In [ ]:
from src.training.hyperparams import find_threshold_if_needed

threshold, validation_accuracy = find_threshold_if_needed(val_loader.dataset.labels, clf.predictions_proba, MODEL_DESCRIPTION)
trainer.test(clf, test_loader)

Threshold: 0.5748395323753357, Validation Accuracy: 0.8468468189239502



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


[{}]

In [ ]:
from src.testing.testing import get_results, write_outputs

results = get_results(clf.predictions_proba, threshold, test_loader.dataset)
outputs = write_outputs(results, MODEL_DESCRIPTION, '/content/gdrive/My Drive/mcl-wic-outputs/ONLY_WIC_', validation_accuracy, threshold)